## First calculate MCMC posterior predictions

In [ ]:
%pylab inline
from skmonaco import mcmiser
from Chempy.parameter import ModelParameters
from Chempy.cem_function import posterior_function_mcmc_quick
from Chempy.score_function import preload_params_mcmc
from Chempy.plot_mcmc import restructure_chain
a = ModelParameters()
preload = preload_params_mcmc()

In [ ]:
from Chempy.wrapper import single_star_optimization
%timeit -r 1 -n 1 single_star_optimization()


In [ ]:
restructure_chain('mcmc/')
positions = np.load('mcmc/posteriorPDF.npy')
init_param = []
for j in range(len(a.p0)):
    init_param.append(np.percentile(positions[:,j],50))
print(init_param)

In [ ]:
# Define function to return posterior predictions (must be exponentiated)
def posterior(theta):
    a = ModelParameters()
    post,_ = posterior_function_mcmc_quick(theta,a,preload)
    posterior = np.exp(post)
    return posterior

sigma = [] # Read prior sigma from parameter file
for i,param_name in enumerate(a.to_optimize):
    sigma.append(a.priors.get(param_name)[1])
sigma = np.array(sigma)

In [ ]:
%%timeit -r 1 -n 1
Nsamples2 = [50000]#,10000,100000] # No MC samples
Nsigma2 = [3.5] # Width in parameter space
i=0
integral2 = []
integral_err2 = []
for j in range(len(Nsamples2)):
    print('Calculating integral %d of %d' %(j+1,len(Nsamples2)))
    result,error = mcmiser(posterior,npoints=Nsamples2[j],xl=init_param-Nsigma2[i]*sigma,xu=init_param+Nsigma2[i]*sigma)
    integral2.append(result)
    integral_err2.append(error)
    print('Result is %.8f +/- %.8f after %d samples (percentage error is %.2f)' %(integral2[-1],integral_err2[-1],Nsamples2[j],100*integral_err2[-1]/integral2[-1]))


In [ ]:
plt.clf()
plt.errorbar(Nsamples2,integral2,yerr=integral_err2,fmt='o')

In [ ]:
ratio = posterior(init_param)/posterior(init_param+sigma)
print(ratio)

In [ ]:
factor = np.sqrt(1./np.log(ratio)*np.dot(sigma,sigma.T)/2)

In [ ]:
d1 = []
d2 = []
d3 = []
from scipy.stats import norm
Nsigma = 3
for i in range(5000):
    param = random.uniform(np.array(init_param)-Nsigma*np.array(sigma),np.array(init_param)+Nsigma*np.array(sigma),size=6)
    param[2] = init_param[2]
    param[3] = init_param[3]
    param[4] = init_param[4]
    param[5] = init_param[5]
    post,_ = posterior_function_mcmc_quick(param,a,preload)
    posterior = np.exp(post)
    d1.append(param[0])
    d2.append(param[1])
    dis = np.sqrt(np.dot(param-init_param,(param-init_param).T))
    d3.append(posterior/norm.pdf(dis,scale=0.18))
    if i%100==0:
        print(i)


In [ ]:
plt.scatter(d1,d2,c=(d3),vmax=0.005)
plt.colorbar()

In [ ]:
from skmonaco import mcimport
skmonaco.mcimport(posterior(theta), npoints, distribution, args=(), dist_kwargs={}, rng=None, nprocs=1, seed=None, batch_size=None, weight=1.0)

In [ ]:
plt.errorbar(Nsigma,integral,yerr=integral_err)

In [ ]:
#factor = scinorm(a.p0,mean=init_param,cov=cov)
init_param

In [ ]:
posterior_mod(init_param+np.array([0.1,0.1,0.1,0.1,0.1,0.1]))

In [ ]:
posterior_mod(init_param)

In [ ]:
Nsigma = 3
d1 = []
d2 = []
d3 = []
for i in range(5000):
    param = random.uniform(np.array(init_param)-Nsigma*np.array(sigma),np.array(init_param)+Nsigma*np.array(sigma),size=6)
    param[2] = init_param[2]
    param[3] = init_param[3]
    param[4] = init_param[4]
    param[5] = init_param[5]
    posterior2 = posterior_mod(param)
    d1.append(param[0])
    d2.append(param[1])
    d3.append(posterior2)
    if i%100==0:
        print(i)


In [ ]:
plt.scatter(d1,d2,c=(d3))
plt.colorbar()

In [ ]:
cov = np.zeros((len(init_param),len(init_param)))
for i in range(len(init_param)):
    param = np.copy(init_param)
    param[i] = init_param[i]+sigma[i] # Change by 1 sigma in this direction
    factor = posterior(np.array(init_param))/posterior(np.array(param))
    cov[i,i] = np.sqrt(sigma[i]**2/(2*np.log(factor)))

In [ ]:
restructure_chain('mcmc/')
positions = np.load('mcmc/posteriorPDF.npy')
init_param = []
for j in range(len(a.p0)):
    init_param.append(np.percentile(positions[:,j],50))
print(init_param)

In [ ]:
%pylab inline
#from skmonaco import mcmiser
from Chempy.parameter import ModelParameters
from Chempy.cem_function import posterior_function_mcmc_quick
from Chempy.score_function import preload_params_mcmc
from Chempy.plot_mcmc import restructure_chain
a = ModelParameters()
preload = preload_params_mcmc()

In [ ]:
sigma = [] # Read prior sigma from parameter file
for i,param_name in enumerate(a.to_optimize):
    sigma.append(a.priors.get(param_name)[1])
sigma = np.array(sigma)

def posterior(theta):
    a = ModelParameters()
    post,_ = posterior_function_mcmc_quick(theta,a,preload)
    posterior = np.exp(post)
    return posterior

Nsigma_width=2
widening = 1.5 # 2x wider gaussian distribution
cov = np.zeros((len(init_param),len(init_param)))
for i in range(len(init_param)):
    param = np.copy(init_param)
    param[i] = np.copy(init_param[i])+Nsigma_width*sigma[i] # Change by 1 sigma in this direction
    factor = posterior(np.array(init_param))/posterior(np.array(param))
    cov[i,i] = (widening*Nsigma_width*sigma[i])**2/(2*np.log(factor)) # np.sqrt()?
# Using 2 sigma at moment, 1.21 factor widens it slightly by 10% to ensure gaussian is wider than posterior

In [ ]:
# Load PDF of posteriors
positions = np.load('mcmc/posteriorPDF.npy')
cov_matrix = np.zeros((len(a.p0),len(a.p0)))
for i in range(len(a.p0)):
    for j in range(len(a.p0)):
        cov_matrix[i,j] = np.cov((positions[:,i],positions[:,j]))[1,0]


In [ ]:
from scipy.stats import multivariate_normal as scinorm
from numpy.random import multivariate_normal as numnorm

# Define function to return posterior predictions (must be exponentiated)

def posterior_mod(theta):
    return posterior(theta)/gauss_factor(theta)

def gauss_factor(theta):
    return scinorm.pdf(theta,mean=np.array(init_param),cov=cov_matrix)

def dist(size):
    # Distribution function
    mean = np.array(init_param)
    return numnorm(mean,cov_matrix,size=size)

# factor is the width of the posterior distribution....

In [ ]:
from skmonaco import mcimport
Nsamples = np.logspace(2,5,4)#,1000, 5000] # No MC samples
#Nsigma = [2,2.5,3,3.5] # Width in parameter space
j = 0
integral = []
integral_err = []

for j in range(len(Nsamples)):
    print('Calculating integral %d of %d' %(j+1,len(Nsamples)))
    result,error = mcimport(posterior_mod,Nsamples[j],dist)
    integral.append(result)
    integral_err.append(error)
print('Final result is %.8f +/- %.8f after %d samples (percentage error is %.2f)' %(integral[-1],integral_err[-1],Nsamples[-1],100*integral_err[-1]/integral[-1]))

In [ ]:
plt.errorbar(np.log10(Nsamples),integral,yerr=integral_err,fmt='x')

In [ ]:
Nsigma = 3
x=[]
y=[]
post=[]
gauss = []
for i in range(2000):
    param = random.uniform(np.array(init_param)-Nsigma*np.array(sigma),np.array(init_param)+Nsigma*np.array(sigma),size=6)
    #param[1] = init_param[1]
    param[2] = init_param[2]
    param[3] = init_param[3]
    param[4] = init_param[4]
    param[5] = init_param[5]
    x.append(param[0])
    y.append(param[1])
    post.append(posterior(param))
    gauss.append(gauss_factor(param))
    if i%500==0:
        print(i)


In [ ]:
plt.clf()
#plt.scatter(x,y,c=np.log10(np.array(post)/np.array(gauss)))
#plt.colorbar()
plt.scatter(x,np.log10(post/max(post)),c='r',s=3)
plt.scatter(x,np.log10(gauss/max(gauss)),c='b',s=3)
#plt.scatter(x,np.log(np.array(post)/np.array(gauss)/max(post)*max(gauss)),c='g')
#plt.xlim([-1.6,-1.5])
#max(post)

In [ ]:
cov_matrix

## Using a truncated gaussian

In [1]:
%pylab inline
#from skmonaco import mcmiser
from Chempy.parameter import ModelParameters
from Chempy.cem_function import posterior_function_mcmc_quick
from Chempy.score_function import preload_params_mcmc
from Chempy.plot_mcmc import restructure_chain
a = ModelParameters()
preload = preload_params_mcmc()

Populating the interactive namespace from numpy and matplotlib


In [2]:
from Chempy.wrapper import single_star_optimization
single_star_optimization()

['Proto-sun_all']
first minimization for each star separately took:  0 seconds
step  1 of  1000
-7.01754609402 -7.01745853971
calculation so far took 1.475217342376709  seconds
step  2 of  1000
-7.01754609402 -7.02027782692
calculation so far took 1.7812283039093018  seconds
step  3 of  1000
-7.01754609402 -7.0205215362
calculation so far took 2.088275194168091  seconds
step  4 of  1000
-7.01754609402 -7.02489725509
calculation so far took 2.408114433288574  seconds
step  5 of  1000
-7.01754609402 -7.02845795305
calculation so far took 2.715280294418335  seconds
step  6 of  1000
-7.01754609402 -7.0462317147
calculation so far took 3.0250515937805176  seconds
step  7 of  1000
-7.01754609402 -7.08557444702
calculation so far took 3.3331546783447266  seconds
step  8 of  1000
-7.01754609402 -7.12624066031
calculation so far took 3.6538338661193848  seconds
step  9 of  1000
-7.01754609402 -7.21159084896
calculation so far took 3.9538345336914062  seconds
step  10 of  1000
-7.01754609402 -7.

-7.01754609402 -9.5583305289
calculation so far took 27.726891040802002  seconds
step  84 of  1000
-7.01754609402 -9.64960514417
calculation so far took 28.025384426116943  seconds
step  85 of  1000
-7.01754609402 -9.71118251217
calculation so far took 28.320210456848145  seconds
step  86 of  1000
-7.01754609402 -9.44075442349
calculation so far took 28.616402864456177  seconds
step  87 of  1000
-7.01754609402 -9.45421527007
calculation so far took 28.915558099746704  seconds
step  88 of  1000
-7.01754609402 -9.31635851676
calculation so far took 29.211674690246582  seconds
step  89 of  1000
-7.01754609402 -9.26678382597
calculation so far took 29.503150463104248  seconds
step  90 of  1000
-7.01754609402 -9.25635279302
calculation so far took 29.792868614196777  seconds
step  91 of  1000
-7.01754609402 -9.06267042551
calculation so far took 30.079935550689697  seconds
step  92 of  1000
-7.01754609402 -9.04838196892
calculation so far took 30.371944904327393  seconds
step  93 of  1000
-

-7.01754609402 -9.75192623096
calculation so far took 52.12574291229248  seconds
step  166 of  1000
-7.01754609402 -9.65024044377
calculation so far took 52.41583442687988  seconds
step  167 of  1000
-7.01754609402 -9.40751254973
calculation so far took 52.70887804031372  seconds
step  168 of  1000
-7.01754609402 -9.5305469913
calculation so far took 53.027859687805176  seconds
step  169 of  1000
-7.01754609402 -9.61389134399
calculation so far took 53.321845293045044  seconds
step  170 of  1000
-7.01754609402 -9.39633434035
calculation so far took 53.60652303695679  seconds
step  171 of  1000
-7.01754609402 -9.54711274044
calculation so far took 53.89552068710327  seconds
step  172 of  1000
-7.01754609402 -9.54823096516
calculation so far took 54.18942713737488  seconds
step  173 of  1000
-7.01754609402 -9.67421589671
calculation so far took 54.475573778152466  seconds
step  174 of  1000
-7.01754609402 -9.73892155291
calculation so far took 54.76890277862549  seconds
step  175 of  100

-7.01754609402 -9.47749092301
calculation so far took 77.59954452514648  seconds
step  249 of  1000
-7.01754609402 -9.31205194174
calculation so far took 77.90341472625732  seconds
step  250 of  1000
-7.01754609402 -9.26719903092
calculation so far took 78.24452781677246  seconds
step  251 of  1000
-7.01754609402 -9.32688043742
calculation so far took 78.58869552612305  seconds
step  252 of  1000
-7.01754609402 -9.45700438254
calculation so far took 79.0118796825409  seconds
step  253 of  1000
-7.01754609402 -9.56464526312
calculation so far took 79.38953351974487  seconds
step  254 of  1000
-7.01754609402 -9.7650531941
calculation so far took 79.69677996635437  seconds
step  255 of  1000
-7.01754609402 -9.94487166418
calculation so far took 80.01593446731567  seconds
step  256 of  1000
-7.01754609402 -9.7522518769
calculation so far took 80.3286190032959  seconds
step  257 of  1000
-7.01754609402 -9.80985699985
calculation so far took 80.65378189086914  seconds
step  258 of  1000
-7.0

[array([-2.39229525, -2.89682338, -0.27253073,  0.56431925,  0.53345018]),
 'initial minimization']

In [3]:
# Load PDF of posteriors
positions = np.load('mcmc/posteriorPDF.npy')
init_param = np.median(positions,axis=0)
cov_matrix = np.zeros((len(a.p0),len(a.p0)))
for i in range(len(a.p0)):
    for j in range(len(a.p0)):
        cov_matrix[i,j] = np.cov((positions[:,i],positions[:,j]))[1,0]

sigma = [] # Read prior sigma from parameter file
for i,param_name in enumerate(a.to_optimize):
    sigma.append(a.priors.get(param_name)[1])
sigma = np.array(sigma)
        
def posterior(theta):
    a = ModelParameters()
    post,_ = posterior_function_mcmc_quick(theta,a,preload)
    posterior = np.exp(post)
    return posterior

from scipy.stats import multivariate_normal as scinorm
from numpy.random import multivariate_normal as numnorm

# Define function to return posterior predictions (must be exponentiated)

def posterior_mod(theta):
    return posterior(theta)/gauss_factor(theta)

def gauss_factor(theta):
    return scinorm.pdf(theta,mean=np.array(init_param),cov=cov_matrix)



In [6]:
# CLIPPED DISTRIBUTION FUNCTION
#Nsigma = 3.

def clipped_dist(size):
    # Distribution function
    #pred = np.zeros(len(init_param))
    output = []
    for j in range(size):
        outside_range=True
        index=1 # Initialisation
        while index != 0:
            index = 0.
            pred = numnorm(init_param,cov_matrix)
            for i in range(len(init_param)): # If in correct range
                if pred[i] > init_param[i]+Nsigma*sigma[i] or pred[i] < init_param[i]-Nsigma*sigma[i]:
                    index += 1
        output.append(pred)
    return np.array(output)
    #numnorm(mean,cov_matrix,size=size)

from scipy.stats import mvn
    
def posterior_clipped_mod(theta):
    low = init_param-Nsigma*sigma # lower limit
    upp = init_param+Nsigma*sigma # upper limit
    mu = np.array(init_param) # mean
    norm,_ = mvn.mvnun(low,upp,init_param,cov_matrix) # Normalisation fraction
    return posterior(theta)/(gauss_factor(theta))*norm


In [ ]:
from skmonaco import mcimport
integral = []
integral_err = []
#Nsigma_list = [1.,1.5,2.,2.5,3.,3.5,4.,4.5]
Nsigma_list = [2.,3.,4.]
for n in range(len(Nsigma_list)):
    Nsigma = Nsigma_list[n]
    print('Calculating integral %d of %d' %(n+1,len(Nsigma_list)))
    result,error = mcimport(posterior_clipped_mod,1e4,clipped_dist)
    integral.append(result)
    integral_err.append(error)
residual = [(integral[-1] - item)/integral[-1]*100 for item in integral]
print('percentage residual',residual)
print('Errors',[integral_err[i]/integral[i]*100 for i in range(len(integral))])

Calculating integral 1 of 3
Calculating integral 2 of 3


In [13]:
print(integral_err[-1]/integral[-1])

0.586387147111


In [ ]:
plt.errorbar(Nsigma_list,(integral),yerr=np.array(integral_err)*1.)    
plt.ylabel('Integral')
plt.xlabel('Range (sigma)')
plt.figure()
residual = [(integral[-1] - item)/integral[-1] for item in integral]
plt.errorbar(Nsigma_list,residual,yerr=integral_err)
plt.ylabel('Fractional Residual')
plt.xlabel('Range (sigma)')

In [ ]:
init_param - Nsigma*sigma

In [ ]:
par=dist(1000)

In [ ]:
x=[]
y=[]
for item in par:
    x.append(item[0])
    y.append(item[1])
_=plt.hist(x,bins=30)

In [ ]:
par2=dist(10000)

In [ ]:
x2=[]
y2=[]
for item in par2:
    x2.append(item[0])
    y2.append(item[1])
plt.hist(x2,bins=30)
plt.hist(x,bins=30)

In [ ]:
Nsigma=1
from scipy.stats import mvn
low = init_param-Nsigma*sigma # lower limit
upp = init_param+Nsigma*sigma # upper limit
mu = np.array(init_param) # mean
frac_enc,_ = mvn.mvnun(low,upp,init_param,cov_matrix) # Fraction enclosed
print(frac_enc)

In [ ]:
# GIVES SAME RESULTS AS ABOVE HENCE WORKS
from math import sqrt
result,err = mcquad(gauss_factor,npoints=1e6,xl=init_param-Nsigma*sigma,xu=init_param+Nsigma*sigma)

In [ ]:
result,err